In [1]:
import configparser
config = configparser.ConfigParser()
config.read('../training/secrets.ini')
import os
import openai
os.environ.update({'OPENAI_API_KEY': config['OPENAI']['OPENAI_API_KEY']})
openai.api_key = os.getenv('OPENAI_API_KEY')

In [2]:
system_prompt_template= """
You are an expert on sparql and wikibase. I have a private wikibase where the p and q items are completely unknown to you. 

If you are asked to provide a "query template json" for a user question, you should respond with a json object with these keys and values:
* 'query_template': a python-style template string. the query template should not use 'rdfs:label' to help you find items. Instead, just use a placeholder tag for each p and q item you need. Each tag should begin with "p-" or "q-" depending on whether you think you need a p or a q item
* 'vocabulary': a list of objects, one for each p or q item you need for a sparql query, each item should have these components:
** 'item_tag': a placeholder string that you make up
** 'item_label_quesses': a list of up to three different strings that you think might be used as the label for that p or q item
Remember: you do not know the actual q or p items in my wikibase, so make sure to list ALL p and q items in the 'vocabulary' section of the json.

You may assume the following prefixes:
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX p: <http://www.wikidata.org/prop/>
PREFIX ps: <http://www.wikidata.org/prop/statement/>

Respond only with the json. Do not include any comments or explanations.

"""

In [3]:
example1_human = """
generate a query template json for the question "Does malin 1 have a right ascension lower than 15.1398?"
"""

In [4]:
example1_ai = """
{{
  "query_template": "SELECT ?result WHERE {{ wd:q-Malin1 wdt:p-RightAscension ?right_ascension . FILTER(?right_ascension < 15.1398) BIND(xsd:boolean(?right_ascension < 15.1398) as ?result) }}",
  "vocabulary": [
    {{
      "item_tag": "q-Malin1",
      "item_label_quesses": ["Malin 1", "Malin-1", "Malin1"]
    }},
    {{
      "item_tag": "p-RightAscension",
      "item_label_quesses": ["Right ascension", "right_ascension", "RA"]
    }}
  ]
}}
"""

In [5]:
example2_human = """
How many children did J.S. Bach have?
"""

In [6]:
example2_ai = """
{{
  "query_template": "w
  "vocabulary": [
    {{
      "item_tag": "q-JSBach",
      "item_label_quesses": ["Johann Sebastian Bach", "Bach, Johann Sebastian", "J.S. Bach"]
    }},
    {{
      "item_tag": "p-Child",
      "item_label_quesses": ["Child", "Offspring", "Progeny"]
    }}
  ]
}}
"""

In [7]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [8]:
template=system_prompt_template
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

example_human_1 = HumanMessagePromptTemplate.from_template(example1_human)
example_ai_1 = AIMessagePromptTemplate.from_template(example1_ai)
example_human_2 = HumanMessagePromptTemplate.from_template(example2_human)
example_ai_2 = AIMessagePromptTemplate.from_template(example2_ai)

human_template="Please generate query template json for this question: {text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [9]:
from langchain.chat_models import ChatOpenAI
chat = ChatOpenAI(temperature=0)

In [10]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, example_human_1, example_ai_1, example_human_2, example_ai_2, human_message_prompt])

In [11]:
from langchain import PromptTemplate, LLMChain
chain = LLMChain(llm=chat, prompt=chat_prompt)
result = chain.run('Which is the Basketball-Reference.com NBA player ID of Hakeem Olajuwon?')
print(result)

{
  "query_template": "SELECT ?id WHERE { wd:q-HakeemOlajuwon wdt:p-BasketballReferencecomNBAPlayerID ?id }",
  "vocabulary": [
    {
      "item_tag": "q-HakeemOlajuwon",
      "item_label_quesses": ["Hakeem Olajuwon", "Olajuwon, Hakeem"]
    },
    {
      "item_tag": "p-BasketballReferencecomNBAPlayerID",
      "item_label_quesses": ["Basketball-Reference.com NBA player ID", "NBA player ID"]
    }
  ]
}


In [12]:
import json

In [13]:
query_template = json.loads(result)

In [14]:
from typing import Any, Dict, List, Optional, Union

def get_nested_value(nested_dict: Dict[str, Any], path: List[str]) -> Optional[Any]:
    """
    Retrieves a value from a nested dictionary structure using a list of keys as a path.

    :param nested_dict: The nested dictionary structure to traverse.
    :param path: A list of strings representing the keys to traverse in order to access the desired value.
    :return: The value at the end of the path if it exists, or None if any key in the path is not found.
    """
    current = nested_dict
    for key in path:
        if not isinstance(current, dict) or key not in current:
            return None
        current = current[key]
    return current

In [15]:
import requests
from typing import Optional

def vocab_lookup(search: str, entity_type: str = "item",
                 item_tag: str = None, 
                 srqiprofile: str = "classic") -> Optional[str]:
    
    if item_tag is not None:
        if item_tag.startswith("q-"):
            entity_type = "item"
        elif item_tag.startswith("p-"):
            entity_type = "property"   
    
    if entity_type == "item":
        srnamespace = 0
    elif entity_type == "property":
        srnamespace = 120
    else:
        raise ValueError("entity_type must be either 'property' or 'item'")      
        
    url = "https://www.wikidata.org/w/api.php"
    params = {
        "action": "query",
        "list": "search",
        "srsearch": search,
        "srnamespace": srnamespace,
        "srlimit": 5,
        "srqiprofile": srqiprofile,
        "format": "json"
    }
    headers = {'Accept': 'application/json'}

    response = requests.get(url, headers=headers, params=params)
        
    if response.status_code == 200:
        results = get_nested_value(response.json(), ['query', 'search'])
        if results and len(results) > 0:
            title = results[0]['title']
            # properties are returned with a prefix 'property:'
            return results[0]['title'].split(':')[-1]
        else:
            print(f"No {entity_type} found for '{search}'")
            return ""
    else:
        print(f"Request failed with status code {response.status_code}")
        return ""

In [16]:
def resolve_query_template(t):
    vocab_items = [(v["item_tag"], v["item_label_quesses"]) for v in t["vocabulary"]]
    # since this is not a proper template, we use "replace" below
    # so we need to be careful that some tags may be prefixes of others
    # sorting them by tag length avoids that
    vocab_items = sorted(vocab_items, key=lambda x: x[0], reverse=True)
    resolved_vocabulary = {item_tag: vocab_lookup(item_label_quesses[0], item_tag=item_tag) for item_tag, item_label_quesses in vocab_items}
    query = t['query_template']
    for item_tag, item_id in resolved_vocabulary.items():
        query = query.replace(item_tag, item_id)
    result = query
    return result

In [17]:
resolve_query_template(query_template)

'SELECT ?id WHERE { wd:Q273256 wdt:P2685 ?id }'

In [18]:
def translator(question):
    llm_result = chain.run(question)
    try:
        query_template = json.loads(llm_result)
    except Exception:
        query_template = "" 
    query = resolve_query_template(query_template)
    return query


In [19]:
translator("The Juventus F.C. has what number of participating teams?")

'SELECT (COUNT(?team) as ?result) WHERE { wd:Q1422 wdt:P1923 ?team }'

# Validation testing for Natural Language Question to SPARQL Query

Create a df for the lcquad queries

In [20]:
import pandas as pd

In [21]:
#lcquad_path = "../../lcquad2.0.train.json"
lcquad_path = "../training/train.json"

sports_indices = {6144, 16386, 26627, 14339, 22531, 8197, 14343, 2059, 6161, 10263, 28696, 12313, 4127, 16417, 12323, 4133, 12328, 20522, 16427, 12333, 10286, 26672, 28724, 10296, 58, 28731, 10298, 61, 16446, 28739, 26692, 10309, 10310, 8262, 10314, 18508, 24652, 10318, 28755, 10324, 24660, 8276, 10328, 18521, 8285, 95, 96, 26723, 16486, 14439, 20587, 16493, 20594, 10357, 26742, 12407, 14457, 12413, 16509, 20606, 20607, 16513, 14466, 24707, 132, 26755, 10374, 10371, 12419, 16523, 4237, 28818, 6135, 6292, 4245, 12439, 22684, 8353, 26787, 20644, 10405, 22692, 12454, 16550, 10409, 16552, 6316, 28846, 26800, 12464, 22707, 16564, 181, 12470, 10425, 14527, 10432, 16576, 201, 10446, 10448, 16594, 6354, 22741, 24795, 20702, 12510, 10465, 14562, 10470, 26855, 231, 234, 24811, 4336, 6386, 28915, 242, 16631, 10488, 2298, 22780, 4349, 28924, 18685, 22785, 18690, 12548, 24839, 28936, 10505, 8458, 4363, 265, 22798, 271, 28943, 24849, 10517, 278, 10519, 26908, 12576, 10538, 28972, 16684, 4399, 14639, 10544, 10548, 8502, 26935, 20792, 24887, 10553, 10552, 10556, 4413, 16702, 16696, 24895, 10561, 321, 16718, 334, 10578, 8532, 12632, 29017, 10584, 4444, 26973, 10589, 348, 26977, 6498, 24930, 16743, 4456, 4460, 24941, 29042, 2420, 12665, 27001, 14716, 27013, 14729, 4491, 16780, 27022, 16783, 10641, 18833, 24982, 407, 16793, 4506, 27035, 24991, 417, 29090, 27042, 29089, 420, 29094, 4519, 12716, 29101, 16813, 430, 6577, 10673, 8628, 16820, 14334, 20920, 14778, 4539, 10689, 27073, 16841, 2506, 20939, 20940, 20941, 20942, 27087, 29136, 464, 29140, 18902, 14807, 27100, 16860, 10721, 25058, 20966, 23015, 14823, 10729, 490, 6636, 27117, 2543, 20976, 4593, 18930, 14836, 16885, 4598, 25077, 4606, 23039, 20992, 14852, 16901, 18948, 519, 16903, 29193, 29194, 4625, 27154, 14869, 23062, 25111, 23061, 27159, 21018, 16918, 21020, 27166, 10783, 12831, 29221, 6694, 29224, 10793, 10794, 14889, 16943, 25136, 25137, 21042, 2611, 4660, 21045, 14906, 12859, 23100, 16956, 16958, 14908, 19010, 27204, 6725, 29260, 10829, 2637, 12881, 29268, 23124, 10838, 16985, 12891, 4700, 27229, 23134, 23135, 6749, 609, 8802, 21090, 25189, 10856, 17003, 2668, 10861, 2669, 17010, 8818, 29299, 629, 14966, 4728, 10876, 17020, 29313, 10883, 17027, 17030, 23175, 2694, 17034, 17042, 15008, 27296, 4768, 29347, 2724, 25253, 4774, 17059, 6822, 19107, 15018, 19114, 29357, 25261, 17073, 4787, 23220, 25271, 19130, 25275, 699, 4796, 22527, 703, 10946, 4804, 15044, 25286, 712, 4813, 10958, 17102, 15056, 17103, 13011, 21205, 21210, 23259, 10976, 8934, 4838, 10984, 29416, 744, 23281, 15091, '0', 2807, 19193, 4858, 763, 4859, 27388, 21248, 770, 13058, 2820, 17156, 11014, 8968, 17162, 15115, 11019, 2827, 2835, 15126, 23319, 15128, 17177, 17178, 4888, 23324, 4896, 17185, 15140, 25383, 13098, 11054, 27439, 29488, 17200, 29494, 17209, 15164, 25404, 2880, 2883, 9029, 2886, 4935, 27466, 4940, 25421, 9038, 17232, 23382, 19291, 2910, 29535, 25438, 25440, 17250, 9058, 29542, 4968, 2920, 29546, 25453, 27504, 883, 2931, 15223, 25465, 4988, 17277, 29564, 11136, 15235, 9092, 27524, 11144, 15240, 21386, 27530, 29578, 2957, 29583, 7060, 17302, 21404, 11165, 2974, 23452, 17312, 21415, 23464, 19369, 2988, 15277, 2989, 5037, 29620, 17333, 2999, 25528, 25535, 3014, 25543, 23497, 11212, 3021, 5071, 7130, 9184, 993, 3042, 29667, 25570, 25573, 3046, 7140, 15336, 13290, 15340, 11246, 5103, 11249, 25587, 11252, 21495, 25592, 29689, 25594, 9208, 1019, 29693, 17405, 27646, 17408, 15361, 9218, 5122, 11268, 25605, 7169, 5121, 3071, 25609, 17418, 17419, 11274, 11277, 11280, 11281, 15379, 25619, 7196, 3101, 21534, 9247, 25633, 23587, 29733, 17447, 3112, 27687, 21543, 23598, 29743, 15413, 5174, 15415, 11317, 7225, 27706, 27711, 25666, 1091, 3140, 17478, 25671, 25673, 17483, 29773, 25678, 15439, 25680, 7249, 11344, 29782, 5208, 11352, 21595, 11356, 13405, 11358, 17505, 19561, 27757, 15472, 25713, 7288, 17530, 3197, 29824, 21633, 15490, 21635, 3204, 17541, 27781, 17543, 15496, 23685, 5259, 27787, 5262, 7310, 5264, 1171, 21655, 23704, 17560, 11417, 15511, 15516, 11423, 21664, 9378, 15524, 21670, 11431, 25770, 11435, 19633, 11444, 23732, 17589, 21689, 9403, 3260, 13505, 9410, 15560, 17609, 15563, 17613, 7375, 25807, 3287, 27863, 17625, 1245, 11486, 17632, 3297, 7394, 21728, 25829, 19686, 7400, 17643, 15597, 29934, 27890, 19699, 25846, 27896, 3321, 11519, 29954, 3334, 29959, 3335, 17673, 13575, 17671, 25868, 25873, 29971, 21780, 27923, 25876, 3351, 9496, 9497, 27929, 11543, 11548, 11565, 1325, 7474, 27955, 19762, 15668, 11575, 11576, 23869, 3391, 1347, 25926, 15687, 11592, 30024, 9544, 23881, 3404, 15688, 7502, 9546, 23886, 13650, 9564, 3422, 3423, 13669, 28008, 25962, 21866, 11628, 11629, 11626, 28013, 5488, 15732, 5493, 3448, 13691, 17789, 17790, 5501, 7552, 13697, 5503, 19843, 25993, 15755, 7564, 3472, 13713, 21907, 13717, 3478, 17818, 30108, 17825, 3489, 3491, 28066, 28072, 17836, 28076, 5551, 7600, 21940, 9653, 26039, 28091, 9665, 30153, 19915, 21964, 11725, 11726, 26070, 13784, 30171, 15835, 7644, 26078, 1499, 19934, 11753, 30188, 19958, 26104, 19962, 3578, 17918, 30206, 28166, 19976, 9736, 13834, 24078, 15886, 3601, 26129, 22035, 26131, 3606, 28185, 28192, 15905, 9763, 24100, 1573, 9768, 28201, 20010, 9771, 11824, 26160, 3635, 11831, 20025, 28220, 5693, 28222, 20032, 28225, 11845, 20040, 7753, 15946, 7755, 3660, 20045, 3665, 15953, 28244, 28249, 15962, 3673, 9824, 3682, 1639, 15976, 28264, 3690, 3691, 3692, 11886, 18031, 5747, 1653, 28281, 26235, 15998, 20099, 13959, 18056, 11913, 1675, 16012, 18060, 18062, 13977, 22171, 22177, 5797, 28331, 5806, 26287, 22194, 24244, 3765, 18101, 7863, 16055, 16056, 28350, 7872, 26308, 20166, 9928, 7882, 14027, 16074, 7885, 22224, 14033, 16082, 20182, 28374, 22243, 22246, 28391, 18152, 20205, 18160, 5873, 20215, 24312, 24313, 12023, 12032, 20228, 3847, 9992, 26380, 20238, 12047, 10006, 26393, 14107, 22301, 20254, 26399, 16172, 16176, 12082, 22324, 16181, 28472, 14146, 14151, 22344, 12114, 28500, 18262, 14166, 28502, 8024, 22366, 22369, 22371, 16227, 5989, 16230, 22376, 26474, 5996, 28526, 24433, 12148, 18292, 24438, 6009, 16250, 14201, 10109, 8063, 26498, 26507, 22415, 28559, 10129, 14225, 22421, 18327, 12184, 12185, 20380, 12190, 6050, 10146, 10148, 28581, 20394, 18349, 20400, 4021, 10166, 12215, 8121, 16319, 20415, 4037, 10182, 6087, 20424, 1991, 10185, 26565, 4050, 14292, 2010, 22491, 18398, 20452, 28649, 22510, 8175, 10224, 16372, 10229, 4087, 22521, 22522, 20476, 24574, 6143}

In [22]:
!ls -l {lcquad_path}

-rwxrwxrwx 1 jstil jstil 26561843 Feb 12 14:22 ../training/train.json


In [25]:
lcquad_df = pd.read_json(lcquad_path)
lcquad_df = lcquad_df.dropna(subset=['uid', 'question', 'sparql_wikidata', 'template_id'])
lcquad_df = lcquad_df[lcquad_df['uid'].isin(sports_indices)]
lcquad_df.head(100)

,NNQT_question,uid,subgraph,template_index,question,sparql_wikidata,sparql_dbpedia18,template,answer,template_id,paraphrased_question
8,What is <International Olympic Committee athle...,26800,center,1097,What's Mary Lou Retton's International Olympic...,select distinct ?answer where { wd:Q235975 wdt...,select distinct ?answer where { ?statement <ht...,E REF ?F,[],1.1,[]
12,Did {Lindsey_Vonn} {participant of} {FIS Alpin...,519,boolean double one_hop right subgraph,19,Did Lindsey Vonn participate in the FIS Alpine...,ASK WHERE { wd:Q10068 wdt:P1344 wd:Q1428251 . ...,ASK { ?statement1 <http://www.w3.org/1999/02/...,Ask (ent-pred-obj1` . ent-pred-obj2),[],2,"In 1999 and 2019, was Lindsey Vonn in the FIS ..."
17,Did {Judi_Dench} {award received} {Tony Award ...,712,boolean double one_hop right subgraph,212,Did Judi Densch receive a Tony Award for best ...,ASK WHERE { wd:Q28054 wdt:P166 wd:Q640374 . wd...,ASK { ?statement1 <http://www.w3.org/1999/02/...,Ask (ent-pred-obj1` . ent-pred-obj2),[],2,Was the Tony Award for best direction of the p...
20,What is {played by} of {succeded by} of {Sleep...,15164,left-subgraph,86,Who Sleepwalking succeeded in playing Sleepwal...,SELECT ?answer WHERE { wd:Q7539946 wdt:P156 ?X...,SELECT ?answer WHERE { ?statement1 <http://www...,C RCD xD . xD RDE ?E,[],5,Who Sleepwalking succeeded in playing Sleepwal...
42,What is the {medal} for {award received} of {A...,29959,simple question right,1550,What is the medal Angela Lansbury recieved?,select distinct ?obj where { wd:Q206856 wdt:P...,select distinct ?obj where { ?statement <http:...,<S P ?O ; ?O instanceOf Type>,[],1,[]
...,...,...,...,...,...,...,...,...,...,...,...
2235,What is <review score> of <12 Angry Men> ?,28244,center,2541,Which is the review score for 12 Angry Men?,select distinct ?answer where { wd:Q2345 wdt:P...,select distinct ?answer where { ?statement <ht...,E REF ?F,[],1.1,[]
2238,What is {member of sports team} of {Michael Jo...,11435,[],903,Who does Michael Jordan play for and how many ...,SELECT ?value1 ?obj WHERE { wd:Q41421 p:P54 ?s...,select distinct ?dataprop1 ?obj where { ?state...,[],[],3,Who does Michael Jordan play for and how numer...
2258,"What is {craft} of {Steve Gerber}, that has {f...",14146,right-subgraph,3851,"In the field of screenplays, what is Steve Ger...",SELECT ?answer WHERE { wd:Q2569995 wdt:P106 ?a...,SELECT ?answer WHERE { ?statement1 <http://www...,E REF ?F . ?F RFG G,[],1,What is Steve Gerber's craft in the screenplay...
2264,what is the {end time} for {Pedro Martinez} ha...,8175,statement_property,4620,When did Pedro Martinez stop playing for the B...,SELECT ?value WHERE { wd:Q726918 p:P54 ?s . ?s...,select distinct ?value where {\n?statement <h...,(E pred ?Obj ) prop value,[],statement_property_1,Give me the date Pedro Martinez stopped playin...


In [24]:
len(lcquad_df)

958

In [ ]:
s_uid = [26800, 15164, 4940, 9496, 26235, 11519, 1245, 2010, 2668, 3197, 25535, 27955, 15439, 10958] 
s_temp = ['1.1', 5, 'statement_property_1', '1', '1.2', 3, 'Count_1', 'Count_2', 'Rank1', 5, 'statement_property_1', '1.1', 5, '1.1']
# 2, statement_property_2, 4, Rank2

In [118]:
templates = ['1.1', 2, 5, 1, 'statement_property_1', '1', 'Rank2',
             'statement_property_2', '1.2', 4, 3, 'Count_1', 'Count_2', 'Rank1']
template_questions = []
template_uids = []

for t in templates:
    row = lcquad_df.loc[lcquad_df['template_id'] == t].iloc[0]
    template_questions.append(row['sparql_wikidata'])
    template_uids.append(row['uid'])

Make a sparql runner

In [41]:
from wikibaseintegrator import wbi_helpers
from wikibaseintegrator.wbi_config import config as wbi_config
import logging

In [42]:
wbi_config['USER_AGENT'] = 'AskwikiBot/1.0 (https://www.wikidata.org/wiki/User:What_Tottles_Meant)'
wbi_config['BACKOFF_MAX_TRIES'] = 1


In [43]:
from requests.exceptions import HTTPError

def run_sparql(query):
    try:
        results = wbi_helpers.execute_sparql_query(query)
    except HTTPError as he:
        logging.error(f'HTTPError {he}')
        print(f"failed query {query}")
        return None
    # print(results)
    if 'boolean' in results:
        return pd.DataFrame([{'Boolean': results['boolean'] }])
    jsonResult = [dict([(k, b[k]['value']) for k in b]) for b in results['results']['bindings']]
    df = pd.DataFrame.from_dict(jsonResult)
    return df


In [44]:
import requests
from typing import List, Dict, Any

def run_sparql(query: str) -> List[Dict[str, Any]]:
    url = 'https://query.wikidata.org/sparql'
    headers = {
        'Accept': 'application/json',
        'User-Agent': 'AskwikiBot/1.0 (https://www.wikidata.org/wiki/User:What_Tottles_Meant)'
    }

    response = requests.get(url, headers=headers, params={'query': query, 'format': 'json'})

    if response.status_code == 200:
        results = response.json().get('results', {}).get('bindings', [])
        return results
    else:
        print(f"Request failed with status code {response.status_code}")
        return None


In [45]:
import time
def validate_queries(qs):
    validation_results = []
    count = 0
    for q in qs:
        print(count)
        if not 'limit' in q.lower():
            q += ' limit 100'
        print(q)
        df = run_sparql(q)
        result_count = 0
        if df is None:
            run_result = 'Fail'
            print(f"Failed query number {count}")
        else:
            run_result = 'Pass'
            result_count = len(df)
        validation_results.append((run_result, result_count))
        count += 1
        time.sleep(2)
    return validation_results 

In [69]:
v = validate_queries(lcquad_df['sparql_wikidata'].loc[0:5])

0
 select distinct ?obj where { wd:Q188920 wdt:P2813 ?obj . ?obj wdt:P31 wd:Q1002697 }  limit 100
1
SELECT ?answer WHERE { wd:Q169794 wdt:P26 ?X . ?X wdt:P22 ?answer} limit 100
2
ASK WHERE { wd:Q174843 wdt:P106 wd:Q1804811 . wd:Q174843 wdt:P106 wd:Q33231 } limit 100
Request failed with status code 400
Failed query number 2
3
SELECT ?answer WHERE { wd:Q675176 wdt:P515 ?X . ?X wdt:P156 ?answer} limit 100
4
select distinct ?answer where { wd:Q32491 wdt:P3362 ?answer} limit 100
5
SELECT DISTINCT ?sbj ?sbj_label WHERE { ?sbj wdt:P31 wd:Q134041 . ?sbj rdfs:label ?sbj_label . FILTER(STRSTARTS(lcase(?sbj_label), 'p')) . FILTER (lang(?sbj_label) = 'en') } LIMIT 25 


In [30]:
v_df = pd.DataFrame(v)

In [31]:
len(v_df[v_df[0] == 'Pass'])

5

    
## Validate the queries

In [51]:
def generate_sparql(question):
    sparql = translator(f"translate English to SPARQL: {question}")
    logging.info(f'sparql {sparql}')
    return sparql


In [33]:
generate_sparql('What are the most common types of liver infection')

"SELECT ?type (COUNT(?disease) AS ?count) WHERE { ?disease wdt:P31 ?type . ?disease rdfs:label ?label . FILTER(LANG(?label) = 'en') FILTER(CONTAINS(LCASE(?label), 'liver infection')) } GROUP BY ?type ORDER BY DESC(?count) LIMIT 10"

In [52]:
def generate_lots_of_sparql(l):
    result = []
    count = 0
    for s in l:
        # if count % 10 == 0:
        print(s) 
        print(count)
        try:
            sp = generate_sparql(s)
        except Exception:
            sp = ""
        result.append(sp)
        count += 1
    print(count)
    return result


In [150]:
%%time
#queries = generate_lots_of_sparql(lcquad_df['question'].loc[50:549])
queries = generate_lots_of_sparql(template_questions)

SELECT ?answer WHERE { wd:Q600751 wdt:P425 ?X . ?X wdt:P461 ?answer}
0
1
CPU times: user 33.5 ms, sys: 2.15 ms, total: 35.7 ms
Wall time: 5.44 s


In [36]:
len(queries)

500

In [40]:
serialized_list = json.dumps(queries)

with open("gpt3_turbo_queries.json", "w") as output_file:
    output_file.write(serialized_list)


In [151]:
%%time
v = validate_queries(queries)

0
SELECT ?answer WHERE { wd:Q600751 wdt:P425 ?X . ?X wdt:P461 ?answer } limit 100
CPU times: user 4.25 ms, sys: 8.7 ms, total: 12.9 ms
Wall time: 2.16 s


In [154]:
print(successful_uid)
print(successful_template)

[26800, 15164, 29959, 4940, 9496, 3197, 6087, 26235, 25535, 11519, 1245, 2010, 2668, 17010]
['1.1', 5, 1, 'statement_property_1', '1', 'Rank2', 'statement_property_2', '1.2', 4, 3, 'Count_1', 'Count_2', 'Rank1', 2]


In [152]:
failed = []
successful = []

for x in range(len(v)):
    if x not in failed:
        successful.append(x)


for i in successful:
    successful_uid.append(template_uids[i])
    successful_template.append(templates[i])
    
    

In [153]:
s = set(successful_template)
total_temp = ['1.1', 2, 5, 1, 'statement_property_1', '1', 'Rank2',
             'statement_property_2', '1.2', 4, 3, 'Count_1', 'Count_2', 'Rank1']
templates = [x for x in total_temp if x not in s]
print(templates, successful_template)

[] ['1.1', 5, 1, 'statement_property_1', '1', 'Rank2', 'statement_property_2', '1.2', 4, 3, 'Count_1', 'Count_2', 'Rank1', 2]


In [149]:
template_questions = []
template_uids = []

for t in templates:
    df = lcquad_df[~lcquad_df.uid.isin(successful_uid)]
    row = df.loc[df['template_id'] == t].iloc[8]
    template_questions.append(row['sparql_wikidata'])
    template_uids.append(row['uid'])

In [56]:
len(v)


14

In [55]:
v_df = pd.DataFrame(v)

In [57]:
len(v_df[v_df[0] == 'Pass'])

9

In [83]:
q = 'SELECT ?facilityLabel WHERE { ?defensive_wall wdt:P846 ?facility . SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . } }'

In [75]:
v_df

,0,1
0,Pass,0
1,Pass,0
2,Pass,0
3,Fail,0
4,Pass,0
...,...,...
495,Pass,0
496,Fail,0
497,Pass,0
498,Pass,0


In [76]:
translator("What is the product with the lowest consumption rate per capita whose has part is paraffin wax?")

TypeError: string indices must be integers

In [77]:
question = "What is the product with the lowest consumption rate per capita whose has part is paraffin wax?"

In [87]:
question = """
When did James Thomas Farrell receive the St. Louis Literary Award?",
"""

In [89]:
query_template_json = chain.run(question)
print(query_template_json)

{
  "query_template": "SELECT ?date WHERE { wd:q-JamesThomasFarrell wdt:p-Award ?award . ?award wdt:p-AwardReceivedDate ?date . ?award wdt:p-AwardReceived ?received . ?received rdfs:label \"St. Louis Literary Award\"@en }",
  "vocabulary": [
    {
      "item_tag": "q-JamesThomasFarrell",
      "item_label_quesses": ["James Thomas Farrell", "Farrell, James Thomas"]
    },
    {
      "item_tag": "p-Award",
      "item_label_quesses": ["Award", "Prize", "Honour"]
    },
    {
      "item_tag": "p-AwardReceivedDate",
      "item_label_quesses": ["Award received date", "Date of receiving award", "Date received"]
    },
    {
      "item_tag": "p-AwardReceived",
      "item_label_quesses": ["Award received", "Receiving award", "Award recipient"]
    }
  ]
}


In [100]:
query_template = json.loads(query_template_json)
resolve_query_template(query_template)

No property found for 'Award received date'


'SELECT ?date WHERE { wd:Q1371154 wdt:P166 ?award . ?award wdt: ?date . ?award wdt:P166 ?received . ?received rdfs:label "St. Louis Literary Award"@en }'

In [97]:
def resolve_query_template(t):
    vocab_items = [(v["item_tag"], v["item_label_quesses"]) for v in t["vocabulary"]]
    vocab_items = sorted(vocab_items, key=lambda x: x[1], reverse=True)
    print(vocab_items)
    resolved_vocabulary = {item_tag: vocab_lookup(item_label_quesses[0], item_tag=item_tag) for item_tag, item_label_quesses in vocab_items}
    print(resolved_vocabulary)
    query = t['query_template']
    for item_tag, item_id in resolved_vocabulary.items():
        query = query.replace(item_tag, item_id)
    result = query
    return result

In [85]:
query_template['vocabulary']

TypeError: string indices must be integers

In [86]:
'limit' in q.lower()

False